In [1]:
import pickle 
import en_core_web_sm
from escopy.text_cleaning_utils import clean_text
from escopy.extract_skills import detect_skills
import pandas as pd 
from tqdm import tqdm

In [2]:

path = 'surface_form_matcher_v02_1.pickle'
model = pickle.load(open(path, "rb"))
nlp = en_core_web_sm.load()


In [3]:
# main 

job_description = """
Advanced skills in SQL, Python, or similar technologies
Experience with Time Series Analysis
Experience with REST APIs and relational databases
Excellent written and verbal communication skills
Ability to clearly document findings and summarise discussions
Drive for excellence, strong attention to detail
Excellent organizational and follow-up skills
Tech-savvy and passionate about new technologies
Bachelor's or Master's Degree required in Computer Science, Mathematics or other related fields.
European Union’s legal working status.
Exposure to AWS cloud services: S3, Athena, Aurora, Elasticsearch, Dynamodb, etc...
Experience with working with database management systems.
"""

# clean text 
job_description = clean_text(job_description)
# extract skill using NESTA skill extractor 
annotations = detect_skills(job_description, model, nlp, return_dict=True, debug=False)


[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# annotation  exploration 
import pprint
print(f'annotations count : {len(annotations)}')
print('annotation example description : ')
pprint.pprint(annotations[0]) 


annotations count : 9
annotation example description : 
{'cluster_0': 0.0,
 'cluster_1': 0.0,
 'cluster_2': 0.0,
 'entity': 1139,
 'label_cluster_0': 'Transversal skills',
 'label_cluster_1': 'General Workplace Skills',
 'label_cluster_2': 'General Workplace Skills',
 'predicted_q': 0.4968150938837877,
 'preferred_label': 'communication',
 'surface_form': 'communication',
 'surface_form_type': 'label_pref'}


# Main code 
run this code to enrich a csv file  

In [17]:
# enrich a csv file of job postings 

job_description_col_name = 'job_description'
print('load file ...')
path = './data/rekrute_data.csv'
data_input = pd.read_csv(path)

print('extract skill ...')
# get job description text list 
job_descriptions = data_input[job_description_col_name].values
# create skill extractor util 
def get_skills(text):
    try : 

        #clean text 
        text = clean_text(text)
        # extract skill using NESTA skill extractor 
        annotations = detect_skills(text, model, nlp, return_dict=True, debug=False)
        # prepare output data [skill_id] (a list of skill ids )
        skill_ids = [str(annotation['entity']) for annotation in annotations]
        
        return skill_ids 

    except : 
        return []



    
# apply 
skills_list = []
for job_description in tqdm(job_descriptions):
    job_skills = get_skills(job_description)
    
    # stringify 
    job_skills_str = ';'.join(job_skills)
    skills_list.append(job_skills_str)

print('merge in input data ...')

output_col_name ='esco_skills'
data_input[output_col_name] = skills_list

success_ratio = len(data_input[data_input[output_col_name]!='']) / len(data_input)
success_ratio = round(success_ratio,2)*100
print('-'*50)
print(f'Done / jobs enriched ({success_ratio} %) ')

print('-'*50)

path_output = './data/rekrute_data_output.csv'
print(f'save data to : {path_output} ')
data_input.to_csv(path_output,index=False)





load file ...
extract skill ...


100%|██████████| 4279/4279 [03:15<00:00, 21.85it/s]


merge in input data ...
--------------------------------------------------
Done / jobs enriched (93.0 %) 
--------------------------------------------------
save data to : ./data/rekrute_data_output.csv 
